In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision.models import MobileNet_V2_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from torch.utils.data import Dataset, DataLoader
# https://www.kaggle.com/datasets/soumikrakshit/nyu-depth-v2

In [ ]:
class DepthwiseUpSampleConvolution(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_depth = nn.Conv2d(in_channels = in_channels, out_channels=in_channels, kernel_size=(5,5), padding=2, groups=in_channels)
        self.bn1 = nn.BatchNorm2d(num_features=in_channels) 
        self.relu6 = nn.ReLU6(inplace=True)
        self.conv_pointwise = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, 1), padding=0)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x  = self.relu6(self.bn1(self.conv_depth(x)))
        return self.relu(self.bn2(self.conv_pointwise(x)))
        
class FastDepth(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobile_v2_encoder = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=MobileNet_V2_Weights.IMAGENET1K_V1)
        self.dec1 = DepthwiseUpSampleConvolution(320, 96)
        self.dec2 = DepthwiseUpSampleConvolution(96, 32)
        self.dec3 = DepthwiseUpSampleConvolution(32, 24)
        self.dec4 = DepthwiseUpSampleConvolution(24, 16)
        self.dec5 = DepthwiseUpSampleConvolution(16, 32)
        self.final_conv = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.values = self.layers_position = {1, 3, 6}

    def forward(self, x):
        # layers = {}
        x = self.mobile_v2_encoder.features[0](x)
        x = self.mobile_v2_encoder.features[1](x)
        layer1 = x
        x = self.mobile_v2_encoder.features[2](x)
        x = self.mobile_v2_encoder.features[3](x)
        layer2 = x
        x = self.mobile_v2_encoder.features[4](x)
        x = self.mobile_v2_encoder.features[5](x)
        x = self.mobile_v2_encoder.features[6](x)
        layer3 = x
        x = self.mobile_v2_encoder.features[7](x)
        x = self.mobile_v2_encoder.features[8](x)
        x = self.mobile_v2_encoder.features[9](x)
        x = self.mobile_v2_encoder.features[10](x)
        x = self.mobile_v2_encoder.features[11](x)
        x = self.mobile_v2_encoder.features[12](x)
        x = self.mobile_v2_encoder.features[13](x)
        x = self.mobile_v2_encoder.features[14](x)
        x = self.mobile_v2_encoder.features[15](x)
        x = self.mobile_v2_encoder.features[16](x)
        x = self.mobile_v2_encoder.features[17](x)
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.dec1(x)

        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.dec2(x)
        x = x + layer3

        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.dec3(x)
        x = x + layer2

        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.dec4(x)
        x = x + layer1
        
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.dec5(x)
        return self.final_conv(x)

In [ ]:
class Conv2NormActivation(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu6 = nn.ReLU6(inplace=True)

    def forward(self, x):
        return self.relu6(self.bn(self.conv(x)))
    
class InvertedResidual(nn.Module):
    def __init__(self, t, in_channels, out_channels, s):
        super().__init__()
        self.conv = nn.Sequential(
            Conv2NormActivation(in_channels, t * in_channels, kernel_size=1, stride=1, padding=0, groups=1), # unchanged data entry
            Conv2NormActivation(t * in_channels, t * in_channels, kernel_size=3, stride=s, padding=1, groups= t * in_channels),
            nn.Conv2d(t * in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )

    def forward(self, x):
        return self.conv(x)

In [ ]:
import csv
import numpy as np

TRAIN_DATA = "./nyu_data/data/nyu2_train.csv"
TEST_DATA = "./nyu_data/data/nyu2_test.csv"
MAX_TRAINING_VALUE = 255
MAX_VALIDATION_VALUE = 10000 # 10 meters expressed in milimiters

class DepthImageDataset(Dataset):
    def __init__(self, csv_file_name, transform=None):
        self.transform = transform
        self.files_paths = []
        self.label_paths = []
        with open(csv_file_name, mode ='r') as file:
            csvFile = csv.reader(file)
            for lines in csvFile:
                self.files_paths.append("./nyu_data/" + lines[0])
                self.label_paths.append("./nyu_data/" + lines[1])

    def __len__(self):
        return len(self.files_paths)
    
    def get_my_item(self, idx):
        return self.__getitem__(idx)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.files_paths[idx], flags=cv2.IMREAD_COLOR)
        depth_img = cv2.imread(self.label_paths[idx], flags=cv2.IMREAD_UNCHANGED)
        if (depth_img.dtype == np.uint8):
            depth_img = depth_img / MAX_TRAINING_VALUE
        else:
            depth_img = depth_img / MAX_VALIDATION_VALUE
        if (self.transform == None):
            return np.array(img, dtype=np.float32), depth_img
        transformed = self.transform(image=np.array(img, dtype=np.float32), mask=depth_img)
        return transformed['image'], transformed['mask'].unsqueeze(dim=0)

In [ ]:
import os
PATH_MODEL = "./trained_fast_depth.pth"
RELATIVE_ERROR = 0.15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_cuda_model(load_data_path: bool):
    model = FastDepth()
    if load_data_path and os.path.isfile(PATH_MODEL):
        print("A citit dintr-ul fisier")
        model.load_state_dict(torch.load(PATH_MODEL, weights_only=True))
    else:
        print('Nu s-a gasit modelul la data path-ul specificat')
    model = model.to(DEVICE)
    print("Device-ul folosite este", DEVICE)
    print("Modelul a fost creat cu succes")
    return model

def get_cpu_model(load_data_path: bool):
    model = FastDepth()
    if load_data_path and os.path.isfile(PATH_MODEL):
        model.load_state_dict(torch.load(PATH_MODEL, weights_only=True))
    else:
        print('Nu s-a gasit modelul la data path-ul specificat')
    print("Device-ul folosite este cpu")
    return model

class EarlyStopping:
    def __init__(self, model : FastDepth, patience, path):
        self.path = path
        self.patience = patience
        self.model = model
        self.current_patience = patience
        self.current_loss = float('inf')

    def should_continue(self, loss):
        if (loss < self.current_loss):
            self.current_loss = loss
            self.current_patience = self.patience
            torch.save(self.model.state_dict(), PATH_MODEL)
            return True
        self.current_patience -= 1
        if (self.current_patience == 0): return False
        return True
    
def plot_losses(train_losses: list[float], validation_losses: list[float], label_x : str, label_y : str, title : str) -> None:
    """Plots training and validation losses over epochs."""
    plt.figure(figsize=(8, 6))
    plt.plot(train_losses, label=label_x, marker="o")
    plt.plot(validation_losses, label=label_y, marker="o")
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_learning_rate(learning_rate_list):
    plt.figure(figsize=(8, 6))
    plt.plot(learning_rate_list, label="LearningRate", marker="o")
    plt.title("Learning Rate curve")
    plt.grid(True)
    plt.show()
    
def compute_depth_accuracy_batch_photos(found_depth: torch.Tensor, depth: torch.Tensor, epsilon=1e-6):
    relative_error = torch.abs(found_depth - depth) / (depth + epsilon) 
    right_examples = relative_error < RELATIVE_ERROR
    total_elements = right_examples.numel()  
    return torch.sum(right_examples).float() / total_elements

def train_depth(model, dataloader, optimizer, criterion):
    model.train()
    train_loss = 0
    train_accuracy = 0 
    idx = 0
    for (inputs, labels) in dataloader:
        idx += 1
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_accuracy += compute_depth_accuracy_batch_photos(outputs, labels).item()
    return train_loss / idx, train_accuracy / idx 

def test_depth(model, dataloader, criterion):
    model.eval()
    with torch.no_grad():
        validation_loss = 0
        validation_accuracy = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            output = model(inputs)
            current_acc = compute_depth_accuracy_batch_photos(output, labels).item()
            validation_loss += criterion(output, labels).item()
            validation_accuracy += current_acc
        return validation_loss / len(dataloader), validation_accuracy / len(dataloader)

In [ ]:
def compute_new_loss_function(found_depth: torch.Tensor, depth: torch.Tensor):
    meanAbsoluteError = torch.mean(torch.abs(found_depth - depth))
    grad_found_x = found_depth[:,:,:-1,:] - found_depth[:,:,1:,:]
    grad_found_y = found_depth[:,:,:,:-1] - found_depth[:,:,:,1:]
    grad_truth_x = depth[:,:,:-1,:] - depth[:,:,1:,:]
    grad_truth_y = depth[:,:,:,:-1] - depth[:,:,:,1:]
    gradientEdgeLoss = torch.mean(torch.abs((grad_found_x - grad_truth_x))) + \
        torch.mean(torch.abs((grad_found_y - grad_truth_y))) 
    mean_found, std_found = found_depth.mean(), found_depth.std()
    mean_truth, std_truth = depth.mean(), depth.std()
    numerator = (2 * mean_found * mean_truth + 1e-5) * (2 * std_found * std_truth + 1e-5)
    denominator =  (mean_found * mean_found + mean_truth * mean_truth + 1e-5) * (std_found * std_found + std_truth * std_truth + 1e-5)
    structuralSimilarityLoss = (1 - numerator / denominator) / 2
    return 0.6 * meanAbsoluteError + 0.2 * gradientEdgeLoss + structuralSimilarityLoss
 
def get_optimized_loss_hyperparameters(model : FastDepth, parameters):
    early_stopper = EarlyStopping(model, parameters["early_stopper"], PATH_MODEL)
    criterion = compute_new_loss_function
    # optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr_optimizer"], weight_decay=parameters["weight_decay_factor"])
    optimizer = torch.optim.SGD(model.parameters(), lr=parameters["lr_optimizer"], momentum=parameters["momentum"], weight_decay=parameters["weight_decay_factor"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=parameters['gamma_lr'], patience=parameters['patience'])
    return early_stopper, criterion, optimizer, lr_scheduler

def get_default_hyperparameters(model :FastDepth, parameters):
    early_stopper = EarlyStopping(model, parameters["early_stopper"], PATH_MODEL)
    criterion = nn.L1Loss()
    optimizer = torch.optim.SGD(model.parameters(), lr=parameters["lr_optimizer"], momentum=parameters["momentum"], weight_decay=parameters["weight_decay_factor"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=parameters['gamma_lr'], patience=parameters['patience'])
    return early_stopper, criterion, optimizer, lr_scheduler

def train_depth_model(data, parameters):
    train_info = {
        "train_loss": [],
        "validation_loss" : [],
        "train_accuracy": [],
        "validation_accuracy" : [],
        "lr": []
    }
    
    max_acc = 0
    model = get_cuda_model(True)
    early_stopper, criterion, optimizer, lr_scheduler = get_optimized_loss_hyperparameters(model, parameters) 
    for i in range(parameters["epochs"]):
        train_loss, train_acc = train_depth(model, data["train"], optimizer, criterion)
        val_loss, val_acc = test_depth(model, data["validation"], criterion)
        if (val_acc > max_acc):
            max_acc = val_acc
            print("acuratete maxima la epoca", i)
        lr_scheduler.step(val_loss)
        print(val_loss, val_acc, "epoch: ", i)
        train_info["train_loss"].append(train_loss)
        train_info["validation_loss"].append(val_loss)
        train_info["train_accuracy"].append(train_acc)
        train_info["validation_accuracy"].append(val_acc)
        # train_info["lr"].append(lr_scheduler.get_lr())
        if not early_stopper.should_continue(val_loss): break
    plot_losses(train_info["train_loss"], train_info["validation_loss"], 
            "Train", "Validation", "Training and Validation Loss")
    plot_losses(train_info["train_accuracy"], train_info["validation_accuracy"], 
            "Train", "Validation", "Training and Validation Accuracy")
    plot_learning_rate(train_info["lr"])
    model.to("cpu")
    return train_info


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 8 epoci pana acum
parameters = {
    "batch_size" : 8,
    'epochs' : 50,
    'lr_optimizer': 1e-2,
    'gamma_lr': 0.3,
    'weight_decay_factor': 1e-4,
    'patience': 30,
    'early_stopper': 5,
    "momentum" : 0.9,
}

SIZE=(256, 320)


train_transforms_no_normalization = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),
    ToTensorV2()
])

train_transforms = A.Compose([ 
    # A.VerticalFlip(p=0.5),
    # A.HorizontalFlip(p=0.5),
    A.Resize(height=SIZE[0], width=SIZE[1]),
    A.Normalize(normalization='min_max'),
    # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),
    A.Normalize(normalization='min_max'),
    # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

datasets = {
    "train_no_transform": DepthImageDataset(TRAIN_DATA, transform=train_transforms_no_normalization), 
    "train": DepthImageDataset(TRAIN_DATA, transform=train_transforms),
    "validation": DepthImageDataset(TEST_DATA, transform=test_transforms)
}

data = {
    "train": DataLoader(datasets['train'], batch_size=parameters["batch_size"], shuffle=True, num_workers=4),
    "validation": DataLoader(datasets['validation'], batch_size=parameters["batch_size"], shuffle=True, num_workers=4)
}
print(len(data['validation'].dataset))
train_depth_model(data, parameters)

In [ ]:
model = get_cpu_model(True)
for iddx, (images, labels) in enumerate(data["validation"]):
    out = model(images)
    print(compute_depth_accuracy_batch_photos(out, labels))
    initial_image = images[0].permute((1, 2, 0))
    labels_image = labels[0].permute((1, 2, 0))
    output_image = out[0].permute((1, 2, 0))
    labels_image = (labels_image.detach().cpu().numpy().squeeze())
    output_image = (output_image.detach().cpu().numpy().squeeze())
    plt.figure(figsize=(24, 24))
    plt.subplot(1, 3, 1) 
    plt.imshow(initial_image)
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(output_image)
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(labels_image)
    plt.axis('off')
    break


In [ ]:
torch_input = torch.rand(size=(1, 3, 480, 640))
model_to_convert = FastDepth()
model_to_convert.load_state_dict(torch.load(PATH_MODEL, weights_only=True))
torch.onnx.export(model_to_convert, torch_input, "fast_depth.onnx", opset_version=11)
import onnxruntime
session = onnxruntime.InferenceSession("./fast_depth.onnx")
print("ONNX model loaded successfully in ONNX Runtime")